In [ ]:
import openai
import os
import random
import re
import time

openai.api_key="******"


In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        #temperature=0.8, 
    )
    return response.choices[0].message["content"]

In [ ]:
game_rule="""You are playing a game called "Who's Undercover," designed for five players divided into two roles: Civilians and Undercover. Each player receives a word, with Undercover players getting a slightly different but similar word compared to Civilians.

The goal is to identify the Undercover players through descriptions and voting. Each round, players describe their word with a single short sentence highlighting one characteristic, avoiding repetition of others' descriptions and without explicitly stating the word. For example, if your word is "orange," you cannot say the word "orange" directly.

After everyone has spoken, all players vote, and the person with the most votes is eliminated. Civilians aim to identify and vote out the Undercover players, while the Undercover players work together to confuse others and vote out Civilians.

The game continues until either all Undercover players are eliminated (Civilians win) or only one Civilian remains (Undercover wins)."""


game_roles=["In this round, there are five players, and you will play as Player 1. Your word is: whale. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 2. Your word is: shark. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 3. Your word is: whale. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 4. Your word is: shark. Please describe your word according to the game rules.",
            "In this round, there are five players, and you will play as Player 5. Your word is: whale. Please describe your word according to the game rules."]

In [4]:
role_order=[1,2,3,4,5]
current_role_index= 0
history=[]
def get_next_role():
    global current_role_index
    # if current_role_index is None: 
    #     next_role=random.choice(role_order)  #random beginning
    # else:
    next_role = role_order[current_role_index]
    current_role_index = (current_role_index + 1) % len(role_order)
    return next_role

## Speaking Session

In [ ]:
#game round   ==Run it before the start of each round==
last_inquiry_time = 0
history_round=[]
def make_response(prompts):
    global history,last_inquiry_time
    role = get_next_role()
    # Check that the time interval is greater than 30 seconds
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
    # Update the last time GPT was asked
    last_inquiry_time = time.time()
    
    # Combine history and current prompt into input text
    
    output=prompts+"This is a record of the previous description, please do not duplicate the description."+str(history)+"""Follow the rules carefully. Think step by step, and hide the word itself in your description! Make your final statement for this round: describe your word in one sentence."""
    response2=get_completion(output)
    print(output)
    print(f'Player{role} said:')
    print(response2)
    history.append(f'player{role} said:{response2}')
    history_round.append(f'player{role} said:{response2}')
    #print(role)

In [ ]:
# ===speaking response===
current_role_index = random.randint(0, len(role_order) - 1) #random beginning

# every round discussion
for _ in range (len(role_order)):
    game=game_rule+game_roles[current_role_index]
    #print(game)
    make_response(game)

## Voting Session

In [ ]:
def response_vote():
    global history,votes,last_inquiry_time
    role= get_next_role()
    # Check that the time interval is greater than 30 seconds
    current_time = time.time()
    if current_time - last_inquiry_time < 30:
        print("The time interval is less than 30 seconds, waiting...")
        time.sleep(30 - (current_time - last_inquiry_time))
    
    # Update the last time GPT was asked
    last_inquiry_time = time.time()
    player_num=f'player{role}'
    player_speech = next(filter(lambda x: player_num in x, history_round), None)
    vote_prompts=game_rule+game_roles[current_role_index-1]+f"""You are player {role}. Here is the speech history: {history_round}. Your speech for this round is: {player_speech}. Following the rules and without voting for yourself, reflect on the following based on the speech history: Do you think you are a civilian or an undercover this round? Whose answer do you find trustworthy? Who do you think is on the opposing side? Who do you want to vote for?"""
    response=get_completion(vote_prompts)
    print(vote_prompts)
    print(f'Player{role} speaking in voting session:')
    print(response) 
    output_vote=game_rule+f"""You are player{role},this is your reflection in the voting session: """+response+"""Based on the final result you got from your reflection,tell me to answer in this sentence format:I vote for player in this round.... Don't repeat other people's descriptions,don't vote for yourself,you can only vote for one person."""
    response2=get_completion(output_vote)
    print(f'Player{role} said:')
    print(response2)
    votes.append(f'player{role} said:{response2}')

In [ ]:
history_round

In [ ]:
# ===voting response===
votes=[]
for _ in range(len(role_order)):
    response_vote()

In [ ]:
# ===voting results===
votes

In [ ]:
# input the undercover's number
role_order=[1,2,3,4,5]
outsider=[2,4]

In [ ]:
def check_and_exclude(response, role_order):
    # catch the number
    match = re.search(r'player\s*(\d+)\s*[\u4e00-\u9fa5]*\s*is undercover', response) 
    global game_roles
   
    captured_number = int(match.group(1))
        
    # Check to see if it is in the player list, if it is then strike it out
    if captured_number in role_order:
        role_order.remove(captured_number)
        print(f'Player{captured_number}is out！')
        remove_num=str(captured_number)
        game_roles = list(filter(lambda x: remove_num not in x, game_roles))
    
    remaining_numbers = [number for number in role_order if number not in outsider]
    remaining_outsider = [number for number in role_order if number in outsider]
    print(remaining_numbers)
    if captured_number not in outsider and len(remaining_numbers) > 1:
        return "The game continues."
    if len(remaining_numbers) == 1:
        return 'Game over! The Undercover wins!'
    if len(remaining_outsider) == 0:
        return 'Game over! The Civilian wins!'
    else:
        return "The game continues."
quit_prompt=f"""Assistant, the player with the most votes describing “I think player is undercover” from each player's mouth will be considered undercover, who do you think is undercover based on the vote type? The votes are: {votes}"""
response=get_completion(quit_prompt)

print(response)

check_and_exclude(response, role_order)

In [ ]:
# Checking the players present

game_roles